<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 4 : Visualisation de vecteurs de mots

## Résumé
Le but de ce laboratoire est de visualiser des vecteurs de mots grâce à des méthodes de réduction de dimensionnalité.  La visualisation permettra de deviner plus facilement le "mot du jour" dans le jeu en ligne [Cémantix](https://cemantix.certitudes.org/).

## 1. Prise en main de word2vec

Le modèle Word2vec contient des _embeddings_ des mots qui sont appris automatiquement à partir de textes.  Chaque mot est donc représenté par un vecteur dans un espace avec plusieurs centaines de dimensions (p.ex. 300 ou 500).  Dans cet espace, les mots de sens ou d'usage proches ont des vecteurs proches (les _embeddings_ capturent la similarité entre mots).  Mais la dimensionnalité de l'espace fait qu'il est difficile de visualiser les vecteurs de mots.

La librairie [Gensim](https://radimrehurek.com/gensim/index.html) permet de charger un modèle Word2vec existant (avec la fonction `load_word2vec_format(...)`) et de manipuler les vecteurs de mots (instances de la classe `KeyedVectors`).  On peut notamment utiliser la fonction `similarity(w1, w2)` qui retourne le cosinus des vecteurs correspondant aux mots `w1` et `w2`, à condition que ceux-ci soient connus du modèle.  On peut aussi utiliser la fonction `most_similar([w])` qui retourne les mots voisins d'un ou plusieurs mots.  La [documentation de KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) fournit des exemples utiles.

Veuillez télécharger l'_un des deux_ modèles word2vec suivants, déjà entraînés, mis à disposition par [J.-Ph. Fauconnier](https://fauconnier.github.io/#data):
- [frWac_no_postag_no_phrase_500_cbow_cut100.bin](https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin) - plus petit (229 Mo), mais pas identique au modèle utilisé pour Cémantix ;
- [frWac_no_postag_phrase_500_cbow_cut10.bin](https://embeddings.net/embeddings/frWac_no_postag_phrase_500_cbow_cut10.bin) - bien plus grand (2 Go) donc assez gourmand en mémoire, mais qui est exactement celui utilisé pour Cémantix.
Veuillez placer le modèle choisi dans le dossier `gensim-data` de votre dossier utilisateur.

In [132]:
# Téléchargement, si nécessaire.
# import wget
# url = 'https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin'
# wget.download(url, path_to_model)

In [133]:
from gensim.models import KeyedVectors

In [134]:
path_to_model = "C:\\Users\\lcsch\\gensim-data\\frWac_no_postag_phrase_500_cbow_cut10.bin"

In [135]:
model = KeyedVectors.load_word2vec_format(path_to_model, binary=True, unicode_errors="ignore")

a. Choisissez deux mots de sens proches `w1` et `w2`, et un autre plus différent noté `w3`. Affichez la similarité selon word2vec (cosinus) entre chaque paire de mots.  Ces valeurs correspondent-elles à vos intuitions ?

In [136]:
from gensim.similarities import Similarity

w1 = 'dauphin'
w2 = 'requin'
w3 = 'goudron'

print('w1-w2:', model.similarity(w1,w2))
print('w2-w3:', model.similarity(w2,w3))
print('w1-w3:', model.similarity(w1,w3))


w1-w2: 0.48375186
w2-w3: 0.057056237
w1-w3: -0.019678146


b. Affichez les 10 mots les plus proches de `w1` selon word2vec avec pour chacun sa similarité avec `w1`. 

In [137]:
model.most_similar(w1)

[('cétacé', 0.5253106355667114),
 ('dauphins', 0.5124589204788208),
 ('mammifère_marin', 0.49912703037261963),
 ('sattaya', 0.4936593770980835),
 ('rorqual', 0.48457515239715576),
 ('lion_poisson_clown', 0.4841044545173645),
 ('requin', 0.4837518632411957),
 ('delphinarium', 0.4817071557044983),
 ('globicéphale', 0.4806098937988281),
 ('tucuxi', 0.4790893793106079)]

c. Quels sont les 15 premiers coefficients du vecteur (_embedding_) du mot `w1` ? Quelle est la dimension de ce vecteur ?  Quelle est la taille du vocabulaire connu du modèle ?  Vous pouvez simplement écrire les commandes répondant à ces questions.

In [138]:
vec = model.get_vector(w1)
print('15 premiers coefficients :', vec[:15])
print('Dimension :', vec.shape)

print('Taille du vocabulaire connu :', len(model.vectors))

15 premiers coefficients : [-0.13417305  0.81857145  1.2199419  -0.18103984 -0.27053598  1.004049
 -2.296227    0.48258477 -0.6281404  -2.0846758   1.8269727  -1.7335479
  0.6532203   0.5872189  -0.8560664 ]
Dimension : (500,)
Taille du vocabulaire connu : 1081995


d. Veuillez écrire une fonction appelée `neighbors` selon les spécifications suivantes.  Cette fonction servira plus loin pour l'affichage.
* input : modèle, liste de mots, nombre de voisins (`topn`) ;
* output : liste de mots voisins de chacun des mots de l'input, représentés par un dictionnaire expliqué ci-après ;
* fonctionnement : pour _chacun_ des mots donnés en input, la fonction teste si le mot est dans le vocabulaire du modèle word2vec (sinon elle ne le considère pas), puis demande au modèle la liste des `topn` mots voisins ; pour chacun de ces mots, la fonction construit un `dict` à 4 champs : 
  - mot voisin
  - similarité
  - mot de départ
  - code de couleur associé au mot de départ : 1, 2, etc.

**Exemple** : si on appelle la fonction avec \['école'\], le début du résultat sera :
```
[{'neighbor': 'scolaire',
  'similarity': 0.7114928364753723,
  'ref_word': 'école',
  'color_code': 1}, ...
```

In [139]:
def neighbors(model, word_list, topn = 5):
    result = list()
    for index, word in enumerate(word_list):
        # Si le mot existe...
        if word in model.key_to_index:
            n_neighbors = model.most_similar(word, topn=topn)
            for neighbor, distance in n_neighbors:
                info = {
                    'neighbor':neighbor,
                    'similarity':distance,
                    'ref_word':word,
                    'color_code': index}
                result.append(info)
    return result

In [140]:
# Tester avec cet appel :
print(neighbors(model, ['chat', 'étudier'], 3))

[{'neighbor': 'minou', 'similarity': 0.6334686875343323, 'ref_word': 'chat', 'color_code': 0}, {'neighbor': 'matou', 'similarity': 0.626587450504303, 'ref_word': 'chat', 'color_code': 0}, {'neighbor': 'petit_chatte', 'similarity': 0.6141418218612671, 'ref_word': 'chat', 'color_code': 0}, {'neighbor': 'étude', 'similarity': 0.6593688726425171, 'ref_word': 'étudier', 'color_code': 1}, {'neighbor': 'analyser', 'similarity': 0.6340749263763428, 'ref_word': 'étudier', 'color_code': 1}, {'neighbor': 'approfondir', 'similarity': 0.5609511733055115, 'ref_word': 'étudier', 'color_code': 1}]


## 2. Affichage simple des voisins d'un mot

Pour commencer, veuillez étudier et exécuter le code fourni en exemple : `Labo4_plotly.ipynb`.  Cela vous montrera comment utiliser l'affichage 2D/3D avec `plotly.js`, et vous permettra de traiter les questions suivantes.

Veuillez écrire et exécuter une fonction qui effectue les opérations suivantes, étant donné un mot en entrée :
* obtenir les vecteurs word2vec du mot et de ses `topn` plus proches voisins ;
* transformer ces vecteurs en 2D par l'ACP ;
* afficher en 2D chaque points correspondant à un mot, avec comme étiquette le mot lui-même.

**Notes:** cette fonction sert de version préliminaire à une fonction plus générique demandée ci-dessous.  Il n'est pas demandé de distinguer le mot entré des mots voisins, dans l'affichage (même type de points).  Il n'est pas nécessaire pour l'instant d'utiliser la fonction `neighbors`.

In [141]:
import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

In [142]:
import pandas as pd

def display_PCA_2D_neighbors(model, word, topn = 5):
    word_vec = model.get_vector(word)

    neighbors = [neighbor_word for neighbor_word, distance in model.most_similar(word, topn=topn)]
    vectors = [model.get_vector(neighbor) for neighbor in neighbors]
    vectors.append(word_vec)
    neighbors.append(word)

    pca = PCA(n_components=2)
    pca_data = pca.fit_transform(vectors)

    plot_data_2 = go.Scatter(
                    x = pca_data[:,0], 
                    y = pca_data[:,1],  
                    text = neighbors,
                    name = 'Mots',
                    textposition = "top center",
                    textfont_size = 10,
                    mode = 'markers+text',
                    marker = {'size': 5, 'opacity': .8, 'color': 2})
    
    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        width = 800, height = 600,
        font = {'family' : 'Arial', 'size' : 10}, # propriétés des caractères
        showlegend = True,
        legend = {'x' : 1, 'y' : 0.5, 'font' : {'color' : 'black'}}) # position et font de la légende
    
    plot_figure = go.Figure(data = plot_data_2, layout = layout)
    plot_figure.show()


In [143]:
# Afficher le mot 'chat' et ses 20 plus proches voisins
# projetés en deux dimensions avec PCA (test de la fonction).
display_PCA_2D_neighbors(model, 'précis', 20)

## 3. Affichage configurable des voisins de plusieurs mots

Le but maintenant est d'écrire une fonction `display_dimred_neighbors` qui affiche sur un seul graphique les mots voisins de chaque mot d'une liste de mots donnés.  Vous utiliserez ici la fonction auxiliaire `neighbors` définie plus haut  Les paramètres suivants seront passés à la fonction `display_dimred_neighbors` :

* `model` - le nom du modèle word2vec
* `word_list` - liste de mots dont on veut afficher les voisins (s'ils existent dans `model`)
* `n_components` - dimensionnalité de l'affichage, 2 ou 3
* `topn` - nombre de voisins à afficher pour chaque mot 
* `method` - **méthode de réduction de dimensionnalité : pca, mds, isomap, tsne, ou umap**
* `n_neighbors`- nombre de voisins considérés par la méthode (applicable à isomap, umap, et tsne (appelé alors `perplexity`)).

Trois tests sont demandés à la question 4, mais vous pouvez en exécuter d'autres et les inclure ici.

In [144]:
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from umap import UMAP # Attention, installer le module nommé 'umap-learn' (avec conda install).

In [145]:
def reduce_dimension(neighbors_data, method, n_components, n_neighbors):
    vectors = np.array([model.get_vector(n) for n in neighbors_data['neighbor']])

    if method == 'pca':
        pca = PCA(n_components=n_components)
        return pca.fit_transform(vectors)
    elif method == 'mds':
        mds = MDS(n_components=n_components)
        return mds.fit_transform(vectors)
    elif method == 'isomap':
        isomap = Isomap(n_neighbors=n_neighbors, n_components=n_components)
        return isomap.fit_transform(vectors)
    elif method == 'tsne':
        tsne = TSNE(perplexity=float(n_neighbors), n_components=n_components)
        return tsne.fit_transform(vectors)
    elif method == 'umap':
        umap = UMAP(n_neighbors=n_neighbors, n_components=n_components)
        return umap.fit_transform(vectors)
    else:
        raise 'unrecognized method'

def display_dimred_neighbors(model, word_list, n_components = 3, topn = 5, method = 'pca', n_neighbors = 5):
    neighbors_raw = neighbors(model, word_list, topn)
    for i, word in enumerate(word_list):
        neighbors_raw.append({'neighbor': word, 'similarity': 1, 'ref_word':word, 'color_code':len(word_list) + i})
    neighbors_data = pd.DataFrame.from_dict(neighbors_raw)

    dimred_data = reduce_dimension(neighbors_data, method, n_components, n_neighbors)

    classes = neighbors_data['ref_word'].value_counts().items()

    plots = []
    for ref_word, _ in classes:
        color = neighbors_data[neighbors_data['ref_word'] == ref_word]['color_code'].iloc[0]
        indices = neighbors_data['ref_word'] == ref_word
        
        plot = None
        if n_components == 2:
            plot = go.Scatter(
                            x = dimred_data[:,0][indices], 
                            y = dimred_data[:,1][indices],  
                            text = neighbors_data['neighbor'][indices],
                            name = ref_word,
                            textposition = "top center",
                            textfont_size = 10,
                            mode = 'markers+text',                            
                            marker = {'size': 10, 'opacity': .8, 'color': color})
        elif n_components == 3:
            plot = go.Scatter3d(
                            x = dimred_data[:,0][indices], 
                            y = dimred_data[:,1][indices],
                            z = dimred_data[:,2][indices],
                            text = neighbors_data['neighbor'][indices],
                            name = ref_word,
                            textposition = "top center",
                            textfont_size = 10,
                            mode = 'markers+text',                            
                            marker = {'size': 10, 'opacity': .8, 'color': color})
        else:
            raise 'too many or too little dimensions specified'
        plots.append(plot)

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        width = 800, height = 600,
        font = {'family' : 'Arial', 'size' : 10}, # propriétés des caractères
        showlegend = True,
        legend = {'x' : 1, 'y' : 0.5, 'font' : {'color' : 'black'}}) # position et font de la légende
    
    plot_figure = go.Figure(data = plots, layout = layout)
    plot_figure.show()

In [146]:
display_dimred_neighbors(model, ['précis', 'goudron'], n_components=3, topn=100, method='tsne', n_neighbors=30)

## 4. Application à l'étude des mots voisins dans word2vec

Veuillez choisir quatre mots (`word_list = [m1, m2, m3, m4]`) de façon à ce que m1 et m2 soient très proches par leur sens ou leur usage, m3 un peu plus éloigné, et m4 très éloigné.  L'objectif de cette question est d'étudier la distribution des mots voisins de ces quatre mots (entre 10 et 50) par une visualisation en 2D ou en 3D des vecteurs de mots.  

En expérimentant avec plusieurs configurations (méthode de réduction de dimensionnalité, valeur de `n_neighbors`, nombre de voisins `topn`), veuillez répondre aux questions suivantes, à l'aide d'un ou plusieurs graphiques par question :

**a.** si on utilise une méthode de réduction de dimensionnalité linéaire (PCA ou MDS métrique), les quatre groupes de mots sont-ils clairement séparés dans word2vec ?

**b.** si on utilise une méthode de réduction de dimensionnalité non-linéaire (Isomap, t-SNE ou UMAP), peut-on mieux mettre en évidence les quatre groupes de mots ?  Quels sont les meilleurs paramètres que vous avez trouvés permettant de bien identifier les quatre ensembles ?

**c.** inversement, trouvez-vous des paramètres qui aboutissent à cinq clusters ? ou à trois ?

In [147]:
# Insérer ici la liste de mots.
word_list = ['baleine', 'poisson', 'lion', 'festival']

topn = 50


In [148]:
# Question 4a
display_dimred_neighbors(model, word_list, topn=topn, n_components=2, method='pca')
display_dimred_neighbors(model, word_list, topn=topn, n_components=2, method='mds')


Pour répondre à la question, on ne peut pas dire que les groupes sont très bien séparés. Si on enlèvait les couleurs, on ne pourrait pas facilement visualiser les groupes.

In [149]:
# Question 4b
neigb_count = int(len(word_list) * topn / 3) # bon estimateur pour choisir le paramètre n_neighbors
display_dimred_neighbors(model, word_list, n_components=2, topn=topn, method='isomap', n_neighbors=neigb_count)
display_dimred_neighbors(model, word_list, n_components=2, topn=topn, method='tsne', n_neighbors=neigb_count)
display_dimred_neighbors(model, word_list, n_components=2, topn=topn, method='umap', n_neighbors=neigb_count)

Oui, ces 3 méthodes permettent une meilleur visualisation des mots (surtout tSNE et UMAP). Pour obtenir de bons résultats, il est important de choisir de bons paramètres. Une valeur `n_neighbors` égale à ~1/3 de la taille du groupe nous a donné de bons résultats.

In [150]:
# Question 4c
neigb_count = 2 # paramètre de mauvaise qualité
display_dimred_neighbors(model, word_list, n_components=2, topn=topn, method='isomap', n_neighbors=neigb_count)
display_dimred_neighbors(model, word_list, n_components=2, topn=topn, method='tsne', n_neighbors=neigb_count)
display_dimred_neighbors(model, word_list, n_components=2, topn=topn, method='umap', n_neighbors=neigb_count)

Avec un nombre de voisins égal à 2, les clusters semblent se diviser en sous clusters encore plus petits. Cela dit, même avec des valeurs égal à 4 ou 5 (donc très petit par rapport au nombre total de points), on obtient d'assez bon résultats de clustering.

Exemple avec n_neighbors = 4 :

In [151]:
display_dimred_neighbors(model, word_list, 2, topn, 'umap', 4)

## 5. Outil d'assistance pour le jeu Cémantix

Le jeu en ligne [Cémantix](https://cemantix.certitudes.org/) (aussi proposé sur le site [Dictaly](https://www.dictaly.com/semantiques/))  demande de deviner le *mot du jour* en l'approchant peu à peu par des mots candidats.  Pour chacun, le système indique pour sa similarité word2vec avec le *mot du jour*, ce qui permet de se rapprocher graduellement de la solution.  Expérimentez d'abord un court instant avec le jeu 😀.

L'objectif de cette question est de visualiser les voisinages de mots, pour vous aider à proposer des mots candidats et trouver plus vite la solution.  La procédure est la suivante :
* essayez trois mots au hasard dans Cémantix
* affichez 20-30 mots voisins de ces trois mots grâce à la fonction `display_dimred_neighbors`
* à l'aide des mots affichés, essayez des mots candidats dans Cémantix
* changez l'affichage en remplaçant les trois mots par de meilleurs mots
* continuez jusqu'à trouver le *mot du jour*

**Notes**
* si vous le souhaitez, vous pouvez utiliser le modèle word2vec identique à celui de Cémantix, qui est frWac_no_postag_phrase_500_cbow_cut10.bin fourni par [J.-Ph. Fauconnier](https://fauconnier.github.io/#data)
* ce modèle fait environ 2.3 Go et contient aussi des expressions de plusieurs mots (séparés par '_')
* vous pouvez choisir d'utiliser le petit ou le grand modèle, sachant qu'avec le premier, les suggestions sont moins pertinentes
* Cémantix ignore les expressions de plusieurs mots présentes dans le grand modèle -- vous pouvez choisir de le faire ou non.

In [152]:
# candidate_words = ['homme', 'rouge', 'étudier']
candidate_words = ['exposition', 'musée','peinture']
display_dimred_neighbors(model, candidate_words, n_components = 2, topn = 30, method = 'pca')

Veuillez noter ici vos observations sur la procédure, et coller un extrait montrant votre meilleure performance au jeu.

![image](résultats-cémantix.jpeg)

**Fin du Labo.**  Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.